A Gaussian process is a collection of random variables, any finite number of which have a joint Gaussian distribution.
Gaussian process regressors are powerful and flexible methods that exploit the correlation between input space points. They provide us not only the mean, but also the uncertainty of the estimation, which in an online problem can help in adressing the exploration-exploitation dilemma.
A Gaussian process is completely specified by its mean and covariance function:

$ f(x) \sim GP(m(x),k(x,x')) $ 

If we have not any prior information about the mean $ m(x) $, we will take it to be zero. The covatiance is given by the kernel function $ E[(f(x)-m(x))(f(x')-m(x'))] = k(x,x') $. The kernel function provides a measure of similarity between two points (how much the points influence each other in the function value estimation). The kernel function needs to be positive-definite and symmetric. A common choice is the "squared exponential kernel":

$ k(x,x') = \theta^2 e^{-\frac{(x-x')^2}{2l^2}} $

where $\theta^2$ is a scale factor and $l$ is the lengthscale (controls the "wiggliness" of the function, tells how close two points have to be to influence each other significantly). The intuition behind this kernel is that variables that are close in the input space are highly correlated, while those far away are uncorrelated. For a single test point $x_*$, the mean and variance prediction is given by:

$ \mu_* = k_*^T[K_N + \sigma_n^2I]^{-1}y $

$ \sigma_*^2 = k(x_*,x_*)- k_{*N}^T[K_N + \sigma_n^2I]^{-1}k_* $ 

where $k_*$ is the covariance vector between the test point and training points, $K_N$ the covarariance matrix, $\sigma_n^2$ the noise variance and $y$ the targets vector.

We want to estimate a function that maps the number of bids (x) to the corresponding expected number of clicks (y).

We define a set of possible bids $ X = {0.10,0.15,...,1.00} $ and a function generating the number of clicks $ n(x) = 100(1-e^{-5x}) $. We then add a Gaussian noise. We will generate samples one by one to show how the GP reduces the uncertainty of its estimation once it collects a new sample.

In [22]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

In [24]:
import sklearn as sk

In [2]:
# function that given a bid, returns the expected number of clicks
def n(x):
  # the real function to estimate
  return (1.0 - np.exp(-5.0*x)) * 100

In [3]:
# function that generates the observation by adding some noise
def generate_observations(x, noise_std):
  return n(x) + np.random.normal(0,noise_std, size = n(x).shape)

In [28]:
n_obs = 50
bids = np.linspace(0.0,1.0,20)
x_obs = np.array([])
y_obs = np.array([])
noise_std = 5.0

# loop in which at each iteration we generate a new observation, fit a GP and plot the function estimated by the GP
for i in range(0,n_obs):
  new_x_obs = np.random.choice(bids,1)
  new_y_obs = generate_observations(new_x_obs,noise_std)

  x_obs = np.append(x_obs,new_x_obs)
  y_obs = np.append(y_obs, new_y_obs)

  # initialize GP and set its parameters
  X = np.atleast_2d(x_obs).T
  Y = y_obs.ravel()

  theta = 1.0
  l = 1.0
  # kernel: product of constant kernel and RBF kernel. To both we pass the value of their variable and its possible range
  kernel = C(theta, (1e-3,1e3)) * RBF(l,(1e-3,1e3))
  # intialize the GP by passing the kernel, the noise variance and flagging the y normalization
  gp = GaussianProcessRegressor(kernel=kernel, alpha=noise_std**2,  n_restarts_optimizer = 10)  # normalize_y=True, gives some issues
  # when we fit the GP, we will find the optimum value for the hyperparameters by maximizing the marginal likelyhood; this operation is done "n_restarts_optimizer" times starting from random points
  
  gp.fit(X,Y)

  # prediction and uncertainty plot
  x_pred = np.atleast_2d(bids).T
  y_pred, sigma = gp.predict(x_pred, return_std = True)

  plt.figure(i)
  plt.plot(x_pred, n(x_pred), 'r:', label = r'$n(x)$')
  plt.plot(X.ravel(), Y, 'ro', label = u'Observed Clicks')
  plt.plot(x_pred, y_pred, 'b-', label = u'Predicted Clicks')
  plt.fill(np.concatenate([x_pred,x_pred[::-1]]),
           np.concatenate([y_pred - 1.96 * sigma , (y_pred + 1.96 * sigma)[::-1]]),
           alpha = .5, fc = 'b', ec = 'None', label = '95% conf interval')
  plt.xlabel('$x')
  plt.ylabel('$n(x)$')
  plt.legend(loc = 'lower right')
  plt.show()

Output hidden; open in https://colab.research.google.com to view.

A good practice with GP is to normalize the data (in our case its not needed on the x and it is built in for the y). Then we need to specify the kernel function, set the kernel parameters (for the squared exponential kernel a common choice is lengthscale $l = 1$ and scale factor (variance) $ \theta = 1 $ ). The the GP is trained and, if necessary, the hyperparameters are estimated from the data by maximizing the marginal likelihood or with other techinques.

As points are collected, the GP approaches to the real function and the uncertainty is reduced. The uncertainity is bigger where there are less observations.

In [23]:
GaussianProcessRegressor.__version__

AttributeError: ignored